In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [38]:
train_data = pd.read_csv('train.csv')

In [39]:
train_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [40]:
train_data.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [41]:
train_data.shape

(15304, 12)

In [42]:
train_data['gender'].value_counts()

Female    9446
Male      5857
Other        1
Name: gender, dtype: int64

In [43]:
train_data['ever_married'].value_counts()

Yes    10385
No      4919
Name: ever_married, dtype: int64

In [44]:
train_data['work_type'].value_counts()

Private          9752
children         2038
Self-employed    1939
Govt_job         1533
Never_worked       42
Name: work_type, dtype: int64

In [45]:
train_data['Residence_type'].value_counts()

Rural    7664
Urban    7640
Name: Residence_type, dtype: int64

In [46]:
train_data['smoking_status'].value_counts()

never smoked       6281
Unknown            4543
formerly smoked    2337
smokes             2143
Name: smoking_status, dtype: int64

In [47]:
train_data.replace({'gender':{'Male':0,'Female':1, 'Other':2}}, inplace=True)
train_data.replace({'ever_married':{'Yes':0,'No':1}}, inplace=True)
train_data.replace({'work_type':{'Private':0,'children':1,'Self-employed':2,'Govt_job':3,'Never_worked':4}}, inplace=True)
train_data.replace({'Residence_type':{'Rural':0, 'Urban':1}}, inplace=True)
train_data.replace({'smoking_status':{'never smoked':0,'Unknown':1,'formerly smoked':2,'smokes':3}}, inplace=True)

In [48]:
train_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,0,28.0,0,0,0,0,1,79.53,31.1,0,0
1,1,0,33.0,0,0,0,0,0,78.44,23.9,2,0
2,2,1,42.0,0,0,0,0,0,103.00,40.3,1,0
3,3,0,56.0,0,0,0,0,1,64.87,28.8,0,0
4,4,1,24.0,0,0,1,0,0,73.36,28.8,0,0


In [49]:
#X = train_data.iloc[:,1:11].values
#y = train_data.iloc[:,11].values
y = train_data["stroke"]
features = ["gender", "age",	"hypertension",	"heart_disease", "ever_married", "work_type", "Residence_type",	"avg_glucose_level", "bmi",	"smoking_status"]
X = pd.get_dummies(train_data[features])

In [50]:
print(X)

       gender   age  hypertension  heart_disease  ever_married  work_type  \
0           0  28.0             0              0             0          0   
1           0  33.0             0              0             0          0   
2           1  42.0             0              0             0          0   
3           0  56.0             0              0             0          0   
4           1  24.0             0              0             1          0   
...       ...   ...           ...            ...           ...        ...   
15299       1  22.0             0              0             1          3   
15300       1  46.0             1              0             0          0   
15301       1  75.0             0              0             0          2   
15302       0  46.0             0              0             0          0   
15303       1  14.0             0              0             1          0   

       Residence_type  avg_glucose_level   bmi  smoking_status  
0         

In [51]:
print(y)

0        0
1        0
2        0
3        0
4        0
        ..
15299    0
15300    0
15301    0
15302    0
15303    0
Name: stroke, Length: 15304, dtype: int64


In [52]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

In [53]:
oversampled_data = pd.concat([X_train, y_train], axis=1).copy()

num_samples = y_train.value_counts()[0] - y_train.value_counts()[1]
new_samples = oversampled_data.query("stroke == 1").sample(num_samples, replace=True, random_state=1)

oversampled_data = pd.concat([oversampled_data, new_samples], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

y_train_oversampled = oversampled_data['stroke']
X_train_oversampled = oversampled_data.drop('stroke', axis=1)

In [54]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train_oversampled,y_train_oversampled)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [55]:
y_pred = classifier.predict(X_test)

In [56]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [57]:
#Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy is %f'% accuracy)

Accuracy is 0.784109


In [58]:
#Precision
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred)
print('Precision is %f'% precision)

Precision is 0.125416


In [59]:
#Recall
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred)
print('Recall is %f'% recall)

Recall is 0.748344


In [60]:
test_data = pd.read_csv('test.csv')

In [61]:
test_data.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64

In [62]:
test_data.replace({'gender':{'Male':0,'Female':1, 'Other':2}}, inplace=True)
test_data.replace({'ever_married':{'Yes':0,'No':1}}, inplace=True)
test_data.replace({'work_type':{'Private':0,'children':1,'Self-employed':2,'Govt_job':3,'Never_worked':4}}, inplace=True)
test_data.replace({'Residence_type':{'Rural':0, 'Urban':1}}, inplace=True)
test_data.replace({'smoking_status':{'never smoked':0,'Unknown':1,'formerly smoked':2,'smokes':3}}, inplace=True)

In [63]:
test_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,15304,1,57.0,0,0,0,0,0,82.54,33.4,1
1,15305,0,70.0,1,0,0,0,1,72.06,28.5,1
2,15306,1,5.0,0,0,1,1,1,103.72,19.5,1
3,15307,1,56.0,0,0,0,3,1,69.24,41.4,3
4,15308,0,32.0,0,0,0,0,0,111.15,30.1,3


In [64]:
#test = test_data.iloc[:,1:].values
fea = ["gender", "age","hypertension","heart_disease", "ever_married", "work_type", "Residence_type","avg_glucose_level", "bmi","smoking_status"]
test = pd.get_dummies(test_data[fea])

In [65]:
print(test)

       gender   age  hypertension  heart_disease  ever_married  work_type  \
0           1  57.0             0              0             0          0   
1           0  70.0             1              0             0          0   
2           1   5.0             0              0             1          1   
3           1  56.0             0              0             0          3   
4           0  32.0             0              0             0          0   
...       ...   ...           ...            ...           ...        ...   
10199       1  27.0             0              0             1          0   
10200       0  49.0             0              0             0          0   
10201       1   3.0             0              0             1          1   
10202       0  31.0             0              0             0          0   
10203       1   2.0             0              0             1          1   

       Residence_type  avg_glucose_level   bmi  smoking_status  
0         

In [66]:
predictions = classifier.predict(test)

In [67]:
output = pd.DataFrame({'id': test_data.id, 'stroke': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
